In [2]:
import pandas as pd
import os
import sys
sys.path.append('../py')
import permute as pm

In [3]:
edges = pd.read_csv('data/hetnet_baseedges.csv')

In [4]:
edges.head(2)

,:START_ID,:END_ID,:TYPE
0,Q71969,Q221668,medical-condition-treated_CmD
1,Q71969,Q246084,medical-condition-treated_CmD


In [6]:
pl = pm.get_pair_lists(edges)

In [15]:
def invert_pairs(pair_list):
     return [(b, a) for (a,b) in pair_list]
    

In [35]:
def calc_overlap(pair_list, invert_list, cutoff=.9):
    count_a, count_b = 0, 0
    
    # if there is a large difference in size, don't bother checking
    if (len(pair_list)/len(invert_list) < cutoff or 
        len(invert_list)/len(pair_list) < cutoff):
        return 0
    
    # Check both dierections
    for pair in pair_list:
        if pair in invert_list:
            count_a += 1
            
    for pair in invert_list:
        if pair in pair_list:
            count_b += 1        
    
    # Return the minimum overlap
    return min(count_a/len(pair_list), count_b/len(invert_list))

In [11]:
edges[':TYPE'].value_counts()[:20]

encodes_GeP                         27164
encoded-by_PeG                      27004
subclass-of_DsD                     10218
physically-interacts-with_PpC        3657
physically-interacts-with_CpP        3596
drug-used-for-treatment_DdC          2912
medical-condition-treated_CmD        2822
has-part_ChC                         2378
genetic-association_GgD              2201
genetic-association_DgG              2181
significant-drug-interaction_CsC     1798
instance-of_CiC                        93
symptoms_DsD                           81
has-cause_DhD                          55
subclass-of_CsC                        40
subclass-of_GsC                        39
different-from_CdC                     31
cause-of_DcD                           23
different-from_DdD                     17
opposite-of_DoD                        16
Name: :TYPE, dtype: int64

In [22]:
calc_overlap(pl['opposite-of_DoD'], invert_pairs(pl['opposite-of_DoD']))

1.0

In [16]:
invert_pairs(pl['opposite-of_DoD'])

[('Q202758', 'Q271993'),
 ('Q5420699', 'Q5376444'),
 ('Q18554630', 'Q18554171'),
 ('Q1193891', 'Q572111'),
 ('Q5376444', 'Q5420699'),
 ('Q1614115', 'Q1320276'),
 ('Q18967803', 'Q18554841'),
 ('Q1320276', 'Q1614115'),
 ('Q177254', 'Q168403'),
 ('Q18554841', 'Q18967803'),
 ('Q18554171', 'Q18554630'),
 ('Q16501', 'Q16499'),
 ('Q271993', 'Q202758'),
 ('Q168403', 'Q177254'),
 ('Q572111', 'Q1193891'),
 ('Q16499', 'Q16501')]

In [23]:
from tqdm import tqdm

In [24]:
pair_list = pl

In [38]:
cutoff = 0.9

invert_list = {key: invert_pairs(value) for key, value in pair_list.items()}
kinds = list(pair_list.keys())


for i, kind in tqdm(enumerate(kinds)):
    for idx in range(i+1, len(kinds)):
        overlap = calc_overlap(pair_list[kind], invert_list[kinds[idx]])
        if overlap > cutoff:
            print(kind, kinds[idx], overlap)

6it [00:00, 13.04it/s]

physically-interacts-with_PpC physically-interacts-with_CpP 0.9797648345638501


10it [00:37,  2.79s/it]

encodes_GeP encoded-by_PeG 0.9935944632601973
genetic-association_DgG genetic-association_GgD 0.9809177646524307


12it [00:37,  2.06s/it]

genetic-association_DgP genetic-association_PgD 1.0


14it [00:37,  1.48s/it]

drug-used-for-treatment_DdC medical-condition-treated_CmD 0.9495192307692307


76it [00:38,  1.98it/s]

drug-used-for-treatment_DdG medical-condition-treated_GmD 1.0
cause-of_GcD has-cause_DhG 1.0
different-from_PdG different-from_GdP 1.0
